In [1]:
import numpy as np
import pennylane as qml
from pennylane.optimize import NesterovMomentumOptimizer

In [2]:
n = 6

In [3]:
backend = qml.device("default.qubit", wires = n)

In [4]:
def layerBlock(wgt):
    for q_i in range(n):
        qml.Rot(wgt[q_i, 0], wgt[q_i, 1], wgt[q_i, 2], wires = q_i)
    
    for q_i in range(n):
        qml.CNOT(wires = [q_i, (q_i+1)%n])

In [5]:
def preparation(x_n):
    qb = [i for i in range(n)]
    qml.BasisState(x_n, wires=qb)

In [6]:
@qml.qnode(backend)
def circuit(weights, x_n = None):
    
    preparation(x_n)
    
    for w in weights:
        layerBlock(w)
    
    return qml.expval(qml.PauliZ(0))

In [7]:
def variationalCircuit(param, x_n = None):
    weights = param[0]
    bias = param[1]
    
    return circuit(weights, x_n = x_n) + bias

In [8]:
def squareLoss(target, predictions):
    loss = 0
    for l, p in zip(target, predictions):
        loss += (l - p) ** 2
    
    loss /= len(target)
    
    return loss

def accuracy(target, predictions):
    loss = 0
    for l, p in zip(target, predictions):
        if abs(l - p) < 1e-5:
            loss += 1
    
    loss /= len(target)
    
    return loss

In [9]:
def cost(param, x_n, y_n):
    predictions = [variationalCircuit(param, x_n = x) for x in x_n]
    return squareLoss(y_n, predictions)

In [10]:
data = np.loadtxt("data/dataset1.txt")

dataset = data[:, :n]
target = data[:, n]

#maps {0, 1} -> {-1, 1}
target = target * 2 - np.ones(len(target))

In [11]:
np.random.seed(0)
blocks = 3
m = 4

initialParams = (0.01 * np.random.randn(blocks, n, m), 0.0)

In [12]:
optimizer = NesterovMomentumOptimizer(0.5)
batchSize = 5

In [13]:
params = initialParams
steps = 15

for i in range(steps):
    
    batchIdx = np.random.randint(0, len(dataset), (batchSize,))
    batchX = dataset[batchIdx]
    batchY = target[batchIdx]
    
    params = optimizer.step(lambda f  :cost(f, batchX, batchY), params)
    
    predictions = [np.sign(variationalCircuit(params, x)) for x in dataset]
    acc = accuracy(target, predictions)
    
    print(
        "Step: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            i + 1, cost(params, dataset, target), acc
        )
    )

Step:     1 | Cost: 1.8010182 | Accuracy: 0.5882353 
Step:     2 | Cost: 3.0289655 | Accuracy: 0.5000000 
Step:     3 | Cost: 1.4702072 | Accuracy: 0.5882353 
Step:     4 | Cost: 1.1865104 | Accuracy: 0.5882353 
Step:     5 | Cost: 1.6427542 | Accuracy: 0.5000000 
Step:     6 | Cost: 1.0048568 | Accuracy: 0.5000000 
Step:     7 | Cost: 1.3500693 | Accuracy: 0.5000000 
Step:     8 | Cost: 1.2632958 | Accuracy: 0.5000000 
Step:     9 | Cost: 0.9777113 | Accuracy: 0.5294118 
Step:    10 | Cost: 1.0377031 | Accuracy: 0.5000000 
Step:    11 | Cost: 1.0084767 | Accuracy: 0.5294118 
Step:    12 | Cost: 1.0109812 | Accuracy: 0.5588235 
Step:    13 | Cost: 0.5671435 | Accuracy: 0.9117647 
Step:    14 | Cost: 0.2277128 | Accuracy: 1.0000000 
Step:    15 | Cost: 0.2271634 | Accuracy: 1.0000000 
